# SpaceX Dash App

We use Jupyter Dash to construct an interactive dashboard from SpaceX launch data.

### Import required libraries

In [1]:
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import plotly.express as px
import warnings

### Read the launch data into a dataframe

In [2]:
spacex_df = pd.read_csv("/Users/XXXX/Documents/IBM Applied Data Science Capstone/week 3 visual analytics dashboards/spacex_launch_dash.csv", index_col=0)

Check out the first few rows of the data and their data types.

In [3]:
spacex_df.head()

,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


Identify the maximum and minimum payload values (to be used in a slider for the dashboard later).

In [4]:
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

### Create the a Jupyter Dash application

In [5]:
app = JupyterDash(__name__)

### Create the app layout and callback functions

In [6]:
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                            options = [
                                                {'label': 'All sites', 'value': 'ALL'},
                                                {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                                {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}
                                            ],
                                            value='ALL',
                                            placeholder='Select a launch site',
                                            searchable=True
                                            ),
                                html.Br(),

                                # Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the 
                                #site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0',
                                                      2500: '2500',
                                                      5000: '5000',
                                                      7500: '7500',
                                                      10000: '10000'},
                                                value=[min_payload, max_payload]),

                                # Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        figure = px.pie(spacex_df,
                     values='class',
                     names='Launch Site',
                     title='Total Successful Launches by Site'
                    )
        return figure
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        figure = px.pie(filtered_df,
                    names='class',
                    title='Total Succesful Launches from site {}'.format(entered_site)
                    )
        return figure

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as 
#output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
             [Input(component_id='site-dropdown', component_property='value'),
             Input(component_id='payload-slider', component_property='value')]
             )
def get_scatter_chart(entered_site, payload_range):
    low, high = payload_range
    payload_range_condition = (spacex_df['Payload Mass (kg)'] > low) & (spacex_df['Payload Mass (kg)'] < high)
    if entered_site == 'ALL':
        figure = px.scatter(spacex_df[payload_range_condition],
                            x='Payload Mass (kg)',
                            y='class',
                            color='Booster Version Category',
                            title='Correlation between payload and success for all sites'
        )
        return figure
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        figure = px.scatter(filtered_df[payload_range_condition],
                            x='Payload Mass (kg)',
                            y='class',
                            color='Booster Version Category',
                            title='Correlation between payload and success for {}'.format(entered_site)
                           )
        return figure

### Run the app

In [7]:
warnings.filterwarnings('ignore')
app.run_server(mode="inline")

### Questions

1. Which site has the largest number of successful launches? 
2. Which site has the highest launch success rate?
3. Which payload range(s) has the highest/lowest launch success rate?
4. Which F9 booster version has the highest/lowest launch success rate?

### Answers

1. KSC LC-39A had 10 successful launches.
2. CCAFS SLC-40 had a launch success rate of 42.9%.
3. Payload range [2500, 3500] has 66.7% launch success rate. Payload range [6000, 75000] has 0% launch success rate. Generally, as the minimum payload increases, the launch success rate tends to decrease.
4. Booster version B5 has a 100% launch success rate (out of 1 launch). Booster version FT has a 65.2% launch success rate (out of 23 launches). Booster version v1.0 has a 0% launch success rate (out of 3 launches).